In [ ]:
import os 
import pandas as pd

In [ ]:
ProjectID = 'govuk-bigquery-analytics'
KEY_DIR = os.getenv("BQ_KEY_DIR")
key_file_path = os.path.join(KEY_DIR, os.listdir(KEY_DIR)[0])

In [ ]:
QUERY = """
 WITH
  session_pages AS (
  SELECT
    CONCAT(fullVisitorId,"-",CAST(visitId AS STRING)) AS sessionId,
    content_id
  FROM (
    SELECT
      fullVisitorId,
      visitId,
      hits.page.pagePath AS pagePath,
      (
      SELECT
        value
      FROM
        hits.customDimensions
      WHERE
        index=4) AS content_id,
      (
      SELECT
        value
      FROM
        hits.customDimensions
      WHERE
        index=2) AS document_type
    FROM
      `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
    CROSS JOIN
      UNNEST(sessions.hits) AS hits
    WHERE
      _TABLE_SUFFIX BETWEEN '{START_DATE}'
      AND '{END_DATE}')
  WHERE
    pagePath != '/'
    AND document_type NOT IN ('document_collection',
      'finder',
      'homepage',
      'license_finder',
      'mainstream_browse_page',
      'organisation',
      'search',
      'service_manual_homepage',
      'service_manual_topic',
      'services_and_information',
      'taxon',
      'topic',
      'topical_event')
    AND content_id NOT IN ('00000000-0000-0000-0000-000000000000', '[object Object]')
  GROUP BY
    sessionId,
    content_id),
  occurrence_counts AS (
  SELECT
    page_1,
    page_2,
    co_occurrences,
    SUM(co_occurrences) OVER (PARTITION BY page_1) AS page_1_occurrences,
    SUM(co_occurrences) OVER (PARTITION BY page_2) AS page_2_occurrences,
    SUM(co_occurrences) OVER () AS total_occurrences
  FROM (
    SELECT
      session_pages_1.content_id AS page_1,
      session_pages_2.content_id AS page_2,
      COUNT(DISTINCT session_pages_1.sessionId) AS co_occurrences
    FROM
      session_pages session_pages_1
    JOIN
      session_pages session_pages_2
    ON
      session_pages_1.sessionId = session_pages_2.sessionId
    WHERE
      session_pages_1.content_id != session_pages_2.content_id
    GROUP BY
      page_1,
      page_2 )
--  WHERE co_occurrences > 1 
    ),
  llr_scores AS (
  SELECT
    page_1,
    page_2,
    k11,
    -- k12,
    -- k21,
    -- k22,
    -- N,
    -- k11_k21,
    -- k11_k12,
    -- k21_k22,
    -- k12_k22,
    -- H_k,
    -- H_rowsums_k,
    -- H_colsums_k,
    2*N*(H_k - H_rowsums_k - H_colsums_k) AS llr_score
  FROM (
    SELECT
      page_1,
      page_2,
      k11,
      -- k12,
      -- k21,
      -- k22,
      N,
      -- k11_k21,
      -- k11_k12,
      -- k21_k22,
      -- k12_k22,
      IF(k11>0,
        k11*LOG(k11/N),
        0) + IF(k12>0,
        k12*LOG(k12/N),
        0) + IF(k21>0,
        k21*LOG(k21/N),
        0) + IF(k22>0,
        k22*LOG(k22/N),
        0) AS H_k,
      k11_k12*LOG(k11_k12/N) + k21_k22*LOG(k21_k22/N) AS H_rowsums_k,
      k11_k21*LOG(k11_k21/N) + k12_k22*LOG(k12_k22/N) AS H_colsums_k
    FROM (
      SELECT
        page_1,
        page_2,
        co_occurrences AS k11,
        page_1_occurrences AS k11_k21,
        page_2_occurrences AS k11_k12,
        page_2_occurrences - co_occurrences AS k12,
        page_1_occurrences - co_occurrences AS k21,
        total_occurrences - page_2_occurrences - page_1_occurrences + co_occurrences AS k22,
        total_occurrences - page_2_occurrences AS k21_k22,
        total_occurrences - page_1_occurrences AS k12_k22,
        total_occurrences AS N
      FROM
        occurrence_counts ) ) )

  -- query to output a few ranked links per link
SELECT
  page_1,
  page_2,
  co_occurrences,
  llr_score,
  rank
FROM (
  SELECT
    page_1,
    page_2,
    k11 AS co_occurrences,
    llr_score,
    RANK() OVER (PARTITION BY page_1 ORDER BY CAST(llr_score AS numeric) DESC) AS rank
  FROM
    llr_scores )
WHERE
  rank < 11
ORDER BY
  page_1,
  rank
"""

In [ ]:
print(QUERY.format(START_DATE='20190221', END_DATE='20190221'))

In [ ]:
# df_llr_recs_1 = pd.io.gbq.read_gbq(
#     QUERY.format(START_DATE='20190218', END_DATE='20190221'),
#                            project_id=ProjectID,
#                            reauth=False,
#                            # verbose=True,
#                            private_key=key_file_path,
#                            dialect='standard')

In [ ]:
DATA_DIR = os.getenv("DATA_DIR")


In [ ]:
# df_llr_recs_1.to_csv(os.path.join(DATA_DIR, 'llr_recs_1.csv.gz'),
#                      compression='gzip', index=False)

In [ ]:
df_llr_recs_1 = pd.read_csv(os.path.join(DATA_DIR, 'llr_recs_1.csv.gz'))

In [ ]:
df_llr_recs_1

In [ ]:
content_json_path = '/Users/suganyasivaskantharajah/Downloads/content.json.gz'

In [ ]:
content_lookup_df = pd.read_csv(
    '/Users/suganyasivaskantharajah/code/govuk-taxonomy-supervised-learning/data/clean_content.csv')

In [ ]:
content_lookup_df.head()

In [ ]:
content_lookup_df = content_lookup_df.set_index('content_id')

In [ ]:
content_lookup_dict = content_lookup_df.to_dict(orient='index')

In [ ]:
df_llr_recs_1.columns

In [ ]:
df_llr_recs_1 = df_llr_recs_1.drop('Unnamed: 0', axis=1)

In [ ]:
df_llr_recs_1.head()

In [ ]:
content_lookup_dict['04a0cc0d-0b9f-45ad-bf57-7c54cbab9df9']

In [ ]:
def get_page_title(content_id):
    try:
        return content_lookup_dict[content_id]['title']
    except KeyError:
        return 'unknown'
def get_page_url(content_id):
    try:
        return f"www.gov.uk{content_lookup_dict[content_id]['base_path']}"
    except KeyError:
        return 'unknown'

In [ ]:
get_page_url('04a0cc0d-0b9f-45ad-bf57-7c54cbab9df9')

In [ ]:
df_llr_recs_1['page_1_url'] = df_llr_recs_1['page_1'].map(get_page_url)

In [ ]:
df_llr_recs_1[df_llr_recs_1['page_1_url'] != 'unknown']

In [ ]:
df_llr_recs_1['page_2_url'] = df_llr_recs_1['page_2'].map(get_page_url)

df_llr_recs_1['page_1_title'] = df_llr_recs_1['page_1'].map(get_page_title)
df_llr_recs_1['page_2_title'] = df_llr_recs_1['page_2'].map(get_page_title)

In [ ]:
df_llr_recs_1.to_csv(os.path.join(DATA_DIR, 'llr_recs_with_links.tsv'),
                     index=False, sep='\t')

In [ ]:
df_llr_recs_1.shape

In [ ]:
df_llr_recs_1[df_llr_recs_1['page_1_url'] != 'unknown'].shape

In [ ]:
int(df_llr_recs_1['rank'][0])

In [ ]:
df_llr_recs_1.query(
    'rank <=6 & page_1_url != "unknown" & page_2_url != "unknown" & co_occurrences>1')[['page_1_url'
                 ,'page_2_url','page_1_title','page_2_title',  'llr_score','rank'
                ]].to_csv('../data/llr_recs_with_linkssubset.csv', index=False)

In [ ]:
df_llr_recs_1.head(10)

